In [1]:
import os, pickle
import numpy as np
import pandas as pd
import librosa

In [3]:
#paths
scale = 'MISTROI'
sub = 'sub-04'
dataset = 'movie10'
tracks_path = f'/home/maellef/DataBase/stimuli/{dataset}/'
data_path = f'/home/maellef/DataBase/{dataset}/Preprocessed_fMRI'

In [4]:
def find_label_value(name, label):
    label += '-' if label[-1] != '-' else label
    startval = name.find(label)+len(label)
    endval = startval
    for i in range(startval, len(name)):
        if name[i] in ['_', '.']:
            break
        endval+=1
    return name[startval:endval]

In [5]:
#structure to store all data pair (wav-fMRI)
df = pd.DataFrame({
})
wavbold = []
#general args
tr = 1.49
hrf_nb_tr = 0
sr=22050

sub_path = os.path.join(data_path, scale, sub)

for filename in os.listdir(sub_path):
    #load bold data
    runpath = os.path.join(sub_path, filename)
    with np.load(runpath) as data:
        x = data['X']
    data.close()
    
    #args necessary to extract infos from run
    sub = sub
    task = find_label_value(filename, 'task')
    category = task[:-2]
    #-----------specific to movie10
    repetition = int(find_label_value(filename, 'run'))
    repetition = repetition if repetition>=0 else np.NaN
    print(category, task, repetition)
    
    #load silenced track
    wav_name = f'{dataset}_{task}.wav'
    wav_path = os.path.join(tracks_path, category, wav_name)
    wav, sr = librosa.load(wav_path, sr=sr)
    
    #bold data for track (WITH HRF delay !!!!!)

    #extract corresponding wav/bold
    bold_segment = x
    wav_segment = wav
    segment_df = pd.Series({
        'category':category,
        'task':task,
        'repetition':repetition
    }).to_frame().T
            
    df = pd.concat([df, segment_df], axis='rows', ignore_index=True)  
    wavbold.append((wav_segment, bold_segment))
df.to_csv(f'{dataset}_{sub}_metadata.tsv', sep='\t')
with open(f'{dataset}_{sub}_pairWavBold', 'wb') as f: 
    pickle.dump(wavbold, f) 

FileNotFoundError: [Errno 2] No such file or directory: '/home/maellef/DataBase/movie10/Preprocessed_fMRI/MISTROI/sub-04'